In [9]:
import sqlite3
import yfinance as yf
import pandas as pd
import numpy as np
from time import sleep
from requests.exceptions import SSLError
from datetime import date

In [2]:
conn = sqlite3.connect('sp500_stocks.db')
cursor = conn.cursor()

# Sectors, industry and sub-industry

In [3]:
mapping = pd.read_csv('gics_mapping.csv')
mapping = mapping.applymap(lambda x: x.strip() if type(x) == str else x)
mapping.head()

,sector_id,sector,industry_id,industry,sub_industry_id,sub_industry
0,1,Energy,101,Energy Equipment & Services,10101,Oil & Gas Drilling
1,1,Energy,101,Energy Equipment & Services,10102,Oil & Gas Equipment & Services
2,1,Energy,102,"Oil, Gas & Consumable Fuels",10201,Integrated Oil & Gas
3,1,Energy,102,"Oil, Gas & Consumable Fuels",10202,Oil & Gas Exploration & Production
4,1,Energy,102,"Oil, Gas & Consumable Fuels",10203,Oil & Gas Refining & Marketing


In [4]:
#Populate sectors
sector_data = mapping[['sector_id', 'sector']].drop_duplicates()
sector_insert_data = []
for i in range(0, sector_data.shape[0]):
    # Ensure sector_id is int and sector is str
    sector_insert_data.append((int(sector_data['sector_id'].iloc[i]), str(sector_data['sector'].iloc[i])))
insert_sectors = """
INSERT OR IGNORE INTO sectors (sector_id, sector) 
VALUES (?, ?)"""

cursor.executemany(insert_sectors, sector_insert_data)

In [5]:
# Populate industries
industry_data = mapping[['sector_id', 'industry_id', 'industry']].drop_duplicates()
industry_insert_data = []
for i in range(0, industry_data.shape[0]):
    industry_insert_data.append((int(industry_data['sector_id'].iloc[i]), int(industry_data['industry_id'].iloc[i]), str(industry_data['industry'].iloc[i])))
insert_industry = """
INSERT OR IGNORE INTO industries (sector_id, industry_id, industry) 
VALUES (?, ?, ?)"""
cursor.executemany(insert_industry, industry_insert_data)

In [6]:
# Populate sub-industries
sub_industry_data = mapping[['sector_id', 'industry_id', 'sub_industry_id', 'sub_industry']].drop_duplicates()
sub_industry_insert_data = []
for i in range(0, sub_industry_data.shape[0]):
    sub_industry_insert_data.append((int(sub_industry_data['sector_id'].iloc[i]), int(sub_industry_data['industry_id'].iloc[i]), int(sub_industry_data['sub_industry_id'].iloc[i]), str(sub_industry_data['sub_industry'].iloc[i])))
insert_sub_industry = """
INSERT OR IGNORE INTO sub_industries (sector_id, industry_id, sub_industry_id, sub_industry) 
VALUES (?, ?, ?, ?)"""
cursor.executemany(insert_sub_industry, sub_industry_insert_data)

In [7]:
cursor.execute("SELECT * FROM sectors")
sectors = cursor.fetchall() 
sectors_df = pd.DataFrame(sectors, columns=['sector_id', 'sector'])
sectors_df

,sector_id,sector
0,1,Energy
1,2,Materials
2,3,Industrials
3,4,Utilities
4,5,Health Care
5,6,Financials
6,7,Consumer Discretionary
7,8,Consumer Staples
8,9,Information Technology
9,10,Communication Services


In [8]:
cursor.execute("SELECT * FROM industries")
industries = cursor.fetchall()
industries_df = pd.DataFrame(industries, columns=['industry_id', 'sector_id', 'industry'])
industries_df

,industry_id,sector_id,industry
0,1,101,Energy Equipment & Services
1,1,102,"Oil, Gas & Consumable Fuels"
2,2,201,Chemicals
3,2,202,Construction Materials
4,2,203,Containers & Packaging
...,...,...,...
68,11,1105,Health Care REITs
69,11,1106,Residential REITs
70,11,1107,Retail REITs
71,11,1108,Specialized REITs


In [9]:
cursor.execute("SELECT * FROM sub_industries")
sub_industries = cursor.fetchall()
sub_industries_df = pd.DataFrame(sub_industries, columns=['sub_industry_id', 'sector_id', 'industry_id', 'sub_industry'])
sub_industries_df

,sub_industry_id,sector_id,industry_id,sub_industry
0,1,101,10101,Oil & Gas Drilling
1,1,101,10102,Oil & Gas Equipment & Services
2,1,102,10201,Integrated Oil & Gas
3,1,102,10202,Oil & Gas Exploration & Production
4,1,102,10203,Oil & Gas Refining & Marketing
...,...,...,...,...
158,11,1108,110805,Data Center REITs
159,11,1109,110901,Diversified Real Estate Activities
160,11,1109,110902,Real Estate Operating Companies
161,11,1109,110903,Real Estate Development


# Stock Information

In [6]:
sp_comp_tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].to_list()
sp_comp_sectors = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['GICS Sector'].to_list()
sp_comp_sub_industries = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['GICS Sub-Industry'].to_list()

In [ ]:
sp_comp_data = []
for ticker,sector, sub_ind in zip(sp_comp_tickers,sp_comp_sectors, sp_comp_sub_industries):
    ticker = ticker.replace('.', '-')  # Replace '.' with '-' for yfinance compatibility
    print(f"Fetching data for {ticker}...")
    stock = yf.Ticker(ticker).info
    sp_comp_data.append((stock['symbol'], stock['shortName'], sector, sub_ind, stock['marketCap'], stock['longBusinessSummary']))

Fetching data for MMM...
Fetching data for AOS...
Fetching data for ABT...
Fetching data for ABBV...
Fetching data for ACN...
Fetching data for ADBE...
Fetching data for AMD...
Fetching data for AES...
Fetching data for AFL...
Fetching data for A...
Fetching data for APD...
Fetching data for ABNB...
Fetching data for AKAM...
Fetching data for ALB...
Fetching data for ARE...
Fetching data for ALGN...
Fetching data for ALLE...
Fetching data for LNT...
Fetching data for ALL...
Fetching data for GOOGL...
Fetching data for GOOG...
Fetching data for MO...
Fetching data for AMZN...
Fetching data for AMCR...
Fetching data for AEE...
Fetching data for AEP...
Fetching data for AXP...
Fetching data for AIG...
Fetching data for AMT...
Fetching data for AWK...
Fetching data for AMP...
Fetching data for AME...
Fetching data for AMGN...
Fetching data for APH...
Fetching data for ADI...
Fetching data for AON...
Fetching data for APA...
Fetching data for APO...
Fetching data for AAPL...
Fetching data f

In [11]:
insert_stock_info = """
INSERT OR IGNORE INTO companies (ticker, name, sector, industry, market_cap, description)
VALUES (?, ?, ?, ?, ?, ?)"""
cursor.executemany(insert_stock_info, sp_comp_data)

## Matching companies sectors and industries to according tables

In [12]:
cursor.execute("SELECT * FROM industries WHERE sector_id = 8")
sectors = cursor.fetchall()
industries_df = pd.DataFrame(sectors, columns=['industry_id', 'sector_id', 'industry'])
industries_df

,industry_id,sector_id,industry
0,8,801,Consumer Staples Distribution & Retail
1,8,802,Beverages
2,8,803,Food Products
3,8,804,Tobacco
4,8,805,Household Products
5,8,806,Personal Care Products


In [13]:
# update_statement = """
#     UPDATE companies
#     SET sector = CASE sector
#         WHEN 'Energy' THEN 1
#         WHEN 'Basic Materials' THEN 2
#         WHEN 'Industrials' THEN 3
#         WHEN 'Utilities' THEN 4
#         WHEN 'Healthcare' THEN 5
#         WHEN 'Financial Services' THEN 6
#         WHEN 'Consumer Cyclical' THEN 7
#         WHEN 'Consumer Defensive' THEN 8
#         WHEN 'Technology' THEN 9
#         WHEN 'Communication Services' THEN 10
#         WHEN 'Real Estate' THEN 11
#         ELSE sector
#     END
# """
update_statement = """ 
    UPDATE companies
    SET sector = (SELECT sector_id FROM sectors WHERE sectors.sector = companies.sector)
    WHERE EXISTS (
    SELECT 1
    FROM sectors
    WHERE sectors.sector = companies.sector
)
"""
cursor.execute(update_statement)

In [14]:
update_statement = """ 
    UPDATE companies
    SET industry = (SELECT sub_industry_id FROM sub_industries WHERE sub_industries.sub_industry = companies.industry)
    WHERE EXISTS (
    SELECT 1
    FROM sub_industries
    WHERE sub_industries.sub_industry = companies.industry
)
"""
cursor.execute(update_statement)

In [15]:
test_query = """
SELECT * FROM companies
ORDER BY market_cap DESC
"""
cursor.execute(test_query)
top_companies = cursor.fetchall()
top_companies_df = pd.DataFrame(top_companies, columns=['ticker', 'name', 'sector', 'industry', 'market_cap', 'description'])
top_companies_df['sector'].unique()

array([ 9, 10,  7,  6,  8,  5,  1,  3,  2,  4, 11], dtype=int64)

In [16]:
top_companies_df

,ticker,name,sector,industry,market_cap,description
0,NVDA,NVIDIA Corporation,9,90602,4.389768e+12,"NVIDIA Corporation, a computing infrastructure..."
1,MSFT,Microsoft Corporation,9,90202,3.981503e+12,Microsoft Corporation develops and supports so...
2,AAPL,Apple Inc.,9,90401,3.017795e+12,"Apple Inc. designs, manufactures, and markets ..."
3,GOOGL,Alphabet Inc.,10,100501,2.362676e+12,Alphabet Inc. offers various products and plat...
4,GOOG,Alphabet Inc.,10,100501,2.362663e+12,Alphabet Inc. offers various products and plat...
...,...,...,...,...,...,...
498,MHK,"Mohawk Industries, Inc.",7,70302,7.411715e+09,"Mohawk Industries, Inc. designs, manufactures,..."
499,EMN,Eastman Chemical Company,2,20105,6.979489e+09,Eastman Chemical Company operates as a special...
500,APA,APA Corporation,1,10202,6.592638e+09,"APA Corporation, an independent energy company..."
501,CZR,"Caesars Entertainment, Inc.",7,70601,5.274677e+09,"Caesars Entertainment, Inc. operates as a gami..."


# Stock prices

In [14]:
# Populate the stock_prices table
sp_comp_price_df = pd.DataFrame(columns=['Date','Close','High','Low','Open','Volume','Ticker'])
count = 0
undownloaded_tickers = []
for ticker in sp_comp_tickers:
    print(f"Downloading price data for {ticker}...")
    ticker = ticker.replace('.', '-')  # Replace '.' with '-' for yfinance compatibility
    dat = yf.download(ticker, period='max', interval='1d', auto_adjust=True,progress=False,timeout=None, multi_level_index=False)
    if dat.empty:
        print(f"No data found for {ticker}. Will retry later...")
        undownloaded_tickers.append(ticker)
    price_df = pd.DataFrame(dat)
    if('Adj Close' in price_df.columns):
        price_df.drop(['Adj Close'], axis=1, inplace=True)
    price_df['Ticker']  = ticker
    price_df.reset_index(inplace=True)
    sp_comp_price_df = pd.concat([sp_comp_price_df, price_df], axis=0)
    count += 1
    print(f"Downloaded {count}/503 tickers.")
    sleep(0.5)

for ticker in undownloaded_tickers:
    print(f"Retrying download for {ticker}...")
    ticker = ticker.replace('.', '-')  # Replace '.' with '-' for yfinance compatibility
    dat = yf.download(ticker, period='max', interval='1d', auto_adjust=True,progress=False,timeout=None, multi_level_index=False)
    if dat.empty:
        print(f"No data found for {ticker}. Skipping.")
        continue
    price_df = pd.DataFrame(dat)
    if('Adj Close' in price_df.columns):
        price_df.drop(['Adj Close'], axis=1, inplace=True)
    price_df['Ticker']  = ticker
    price_df.reset_index(inplace=True)
    sp_comp_price_df = pd.concat([sp_comp_price_df, price_df], axis=0)
    count += 1
    print(f"Downloaded {ticker}.")


Downloaded 1/503 tickers.
Downloaded 2/503 tickers.
Downloaded 3/503 tickers.
Downloaded 4/503 tickers.
Downloaded 5/503 tickers.
Downloaded 6/503 tickers.
Downloaded 7/503 tickers.
Downloaded 8/503 tickers.
Downloaded 9/503 tickers.
Downloaded 10/503 tickers.
Downloaded 11/503 tickers.
Downloaded 12/503 tickers.
Downloaded 13/503 tickers.
Downloaded 14/503 tickers.
Downloaded 15/503 tickers.
Downloaded 16/503 tickers.
Downloaded 17/503 tickers.
Downloaded 18/503 tickers.
Downloaded 19/503 tickers.
Downloaded 20/503 tickers.
Downloaded 21/503 tickers.
Downloaded 22/503 tickers.
Downloaded 23/503 tickers.
Downloaded 24/503 tickers.
Downloaded 25/503 tickers.
Downloaded 26/503 tickers.
Downloaded 27/503 tickers.
Downloaded 28/503 tickers.
Downloaded 29/503 tickers.
Downloaded 30/503 tickers.
Downloaded 31/503 tickers.
Downloaded 32/503 tickers.
Downloaded 33/503 tickers.
Downloaded 34/503 tickers.
Downloaded 35/503 tickers.
Downloaded 36/503 tickers.
Downloaded 37/503 tickers.
Downloaded

In [15]:
today = date.today().strftime("%Y-%m-%d")


In [18]:
sp_comp_price_df = sp_comp_price_df[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
sp_comp_price_df['Date'] = pd.to_datetime(sp_comp_price_df["Date"]).dt.strftime("%Y-%m-%d")
sp_comp_price_df = sp_comp_price_df.loc[sp_comp_price_df['Date'] < today]
conn.executemany(
    "INSERT OR IGNORE INTO price (ticker, date, open, high, low, close, volume) VALUES (?, ?, ?, ?, ?, ?, ?)",
    sp_comp_price_df.values.tolist()
)

In [19]:
cursor.execute("SELECT COUNT(DISTINCT ticker) FROM price")
count = cursor.fetchone()
count[0]

503

In [20]:
cursor.execute("SELECT * FROM price WHERE Ticker = 'AAPL' ORDER BY Date DESC")
count = cursor.fetchall()
count_df = pd.DataFrame(count, columns=['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [21]:
# Commit changes and close the connection
conn.commit()
conn.close()